In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import GPy, scipy

# Basics

In [ ]:
GPy.models.GPRegression?

In [ ]:
GPy.kern.RBF?

# Gaussian Processes for Bayesian hypothesis tests on regression functions

## Benavoli et al., 2015

In [ ]:
def equalityTest(x,y1,y2):

    n = x.shape[0]

    xgp = np.zeros((n*2, 2))
    xgp[:,0] = np.tile(x,2)
    xgp[n:,1] = 1

    # input must be 2 dimensional
    ygp = np.concatenate((y1,y2))[:,None]

    kern = GPy.kern.IndependentOutputs(GPy.kern.RBF(1))
    m = GPy.models.GPRegression(xgp,ygp, kern)
    m.randomize()
    m.optimize()

    xpred = np.zeros((n*2,2))
    xpred[:,0] = np.tile(np.sort(x),2)
    xpred[n:,1] = 1

    mu,cov = m.predict_noiseless(xpred,full_cov=True)

    op = np.zeros((n,n*2))
    for i in range(n):
        op[i,i] = 1
        op[i,i+n] = -1

    mu = np.dot(op,mu)
    cov = np.dot(op, np.dot(cov, op.T))

    evals = np.linalg.eigvals(cov)
    nu = sum(evals > 1e-3)

    alpha = scipy.stats.chi2.cdf(np.dot(mu[:,0], np.dot(cov, mu[:,0])), nu)

    return alpha

In [ ]:
# list of function comparisons as described in the manuscript
# functions = [[f1, f2], ...]
# f2 = None implies f2=f1

functions = \
    [[lambda x: 1, None],
     [lambda x: np.exp(x), None],
     [lambda x: np.sin(2*np.pi*x), None],
     [lambda x: 1, lambda x: 1+x],
     [lambda x: np.exp(x), lambda x: np.exp(x) + x],
     [lambda x: np.sin(2*np.pi*x), lambda x: np.sin(2*np.pi*x) + x],
     [lambda x: 1, lambda x: 1 + np.sin(2*np.pi*x)],
     [lambda x: np.exp(x), lambda x: np.exp(x) + np.sin(2*np.pi*x)],
     [lambda x: np.sin(2*np.pi*x), lambda x: 2*np.sin(2*np.pi*x)]
    ]

In [ ]:
# examples of function outputs

n = 50
x = np.random.uniform(size=n)
sigma1 = .01
sigma2 = .01

plt.figure(figsize=(12,12))

for i, funcs in enumerate(functions):
    f1,f2 = funcs
    
    if f2 is None:
        f2 = f1
    
    plt.subplot(3,3,i+1)
    plt.title("test %d" % (i+1))
    plt.scatter(x, f1(x)+np.random.normal(scale=np.sqrt(sigma1),size=n))
    plt.scatter(x, f2(x)+np.random.normal(scale=np.sqrt(sigma2),size=n))

### Test for equality 

In [ ]:
xgp = np.zeros((n*2, 2))
xgp[:,0] = np.tile(x,2)
xgp[n:,1] = 1

plt.plot(xgp)

In [ ]:
# independent outputs kernel allows for different functions 
# to be modeled with a shared background kernel.
#
# input is assumed to be of the form [x, index] where index
# specifies which function each observation belongs to

kern = GPy.kern.IndependentOutputs(GPy.kern.RBF(1))

plt.imshow(kern.K(xgp))

In [ ]:
i = 6

f1,f2 = functions[i]

if f2 is None:
    f2 = f1

y1 = f1(x)+np.random.normal(scale=np.sqrt(sigma1),size=n)
y2 = f2(x)+np.random.normal(scale=np.sqrt(sigma2),size=n)

# input must be 2 dimensional
ygp = np.concatenate((y1,y2))[:,None]

# plt.plot(ygp)
plt.scatter(x,y1)
plt.scatter(x,y2)

In [ ]:
kern = GPy.kern.IndependentOutputs(GPy.kern.RBF(1))
m = GPy.models.GPRegression(xgp,ygp, kern)
m.randomize()
m.optimize()

mnull = GPy.models.GPRegression(xgp,ygp, GPy.kern.RBF(1))
mnull.randomize()
mnull.optimize()

m.log_likelihood() - mnull.log_likelihood()

In [ ]:
m = GPy.models.GPRegression(xgp,ygp, kern)
m.randomize()
m

In [ ]:
m.optimize()
m

In [ ]:
xpred = np.zeros((n*2,2))
xpred[:,0] = np.tile(np.sort(x),2)
xpred[n:,1] = 1

# predict noiseless to get function (not observation) intervals
mu,cov = m.predict_noiseless(xpred,full_cov=True)
cov[cov<1e-9] = 1e-9

std = np.sqrt(np.diagonal(cov))

plt.plot(xpred[:n,0],mu[:n,0])
plt.fill_between(xpred[:n,0],mu[:n,0]-1.98*std[:n],mu[:n,0]+1.98*std[:n],alpha=.1)

plt.plot(xpred[:n,0],mu[n:,0])
plt.fill_between(xpred[:n,0],mu[n:,0]-1.98*std[n:],mu[n:,0]+1.98*std[n:],alpha=.1)

# plt.plot(xpred[:,0],mu[:,0])

In [ ]:
# we build an operation to compute the difference between f1 and f2 at each timepoint t

op = np.zeros((n,n*2))
for i in range(n):
    op[i,i] = 1
    op[i,i+n] = -1
    
plt.imshow(op)

In [ ]:
plt.imshow(cov)

In [ ]:
mu = np.dot(op,mu)
cov = np.dot(op, np.dot(cov, op.T))

In [ ]:
# compute the degrees of freedom based off 
# the number of positive eigenvalues

evals = np.linalg.eigvals(cov + 1e-10*np.eye(cov.shape[0]))
# nu = sum(evals > 1e-9)
nu = sum(evals/sum(evals) > 1e-2)
nu

In [ ]:
std = np.sqrt(np.diagonal(cov))

plt.plot(xpred[:n,0],mu[:n,0])
plt.fill_between(xpred[:n,0],mu[:n,0]-1.98*std[:n],mu[:n,0]+1.98*std[:n],alpha=.1)
plt.axhline(0, color='k')

In [ ]:
plt.imshow(cov)
plt.colorbar()

In [ ]:
cov

In [ ]:
np.dot(mu[:,0], np.dot(cov, mu[:,0]))

In [ ]:
scipy.stats.chi2.cdf(np.dot(mu[:,0], np.dot(cov, mu[:,0])), df=nu)

In [ ]:
i = 6
lls = []

f1,f2 = functions[i]
if f2 is None:
    f2 = f1

for _ in range(100):
    
    y1 = f1(x)+np.random.normal(scale=np.sqrt(sigma1),size=n)
    y2 = f2(x)+np.random.normal(scale=np.sqrt(sigma2),size=n)
    
    ygp = np.concatenate((y1,y2))[:,None]
    
    kern = GPy.kern.IndependentOutputs(GPy.kern.RBF(1))
    m = GPy.models.GPRegression(xgp,ygp, kern)
    m.randomize()
    m.optimize()

    mnull = GPy.models.GPRegression(xgp,ygp, GPy.kern.RBF(1))
    mnull.randomize()
    mnull.optimize()

    lls.append(m.log_likelihood() - mnull.log_likelihood())

In [ ]:
plt.hist(lls)

In [ ]:
for i, funcs in enumerate(functions):
    f1,f2 = funcs
    
    if f2 is None:
        f2 = f1
    
    lls = []
    for _ in range(100):
    
        y1 = f1(x)+np.random.normal(scale=np.sqrt(sigma1),size=n)
        y2 = f2(x)+np.random.normal(scale=np.sqrt(sigma2),size=n)

        ygp = np.concatenate((y1,y2))[:,None]

        kern = GPy.kern.IndependentOutputs(GPy.kern.RBF(1))
        m = GPy.models.GPRegression(xgp,ygp, kern)
        m.randomize()
        m.optimize()

        mnull = GPy.models.GPRegression(xgp,ygp, GPy.kern.RBF(1))
        mnull.randomize()
        mnull.optimize()

        lls.append(m.log_likelihood() - mnull.log_likelihood())
    
    plt.subplot(3,3,i+1)
    plt.title("test %d" % (i+1))
    plt.hist(lls)
    
plt.tight_layout()